In [1]:
from dvc.api import DVCFileSystem
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
import geopandas as geo_pd

fs = DVCFileSystem("https://github.com/tjrileywisc/waltham_etl")


In [2]:

# mbta required
MANDATE_REQD = 3_982

MFH_CITYWIDE = 15_294

# claimed by mayor's office
CARTER_ST_CLAIMED = 5_008
BRANDEIS_CLAIMED = 540

# use codes counted as MFH
MFH_CODES = [
    "112",
    "013",
    "031",
    "043",
    "125",
    "111",
    "109",
    "908",
    "121",
    "105",
    "104",
    "014",
    "041",
    "102",
    "905",
    "920"
]

## Load current zoning and DHCD detail map for Waltham

In [43]:

with fs.open("data/mbta_communities/308_WALTHAM_detail.zip") as f:
    content = BytesIO(f.read())
    zip = ZipFile(content)
    zip.extractall("../../data/mbta_communities/308_WALTHAM_detail")
    property_shapefiles_df = geo_pd.read_file("../../data/mbta_communities/308_WALTHAM_detail/308_WALTHAM_detail.shp")
    
print(property_shapefiles_df)

with fs.open("data/mbta_communities/Transit_Station_Areas_Half_Mile_Radius.zip") as f:
    content = BytesIO(f.read())
    zip = ZipFile(content)
    zip.extractall("../../data/mbta_communities/Transit_Station_Areas_Half_Mile_Radius")
    half_mile_radius_df = geo_pd.read_file("../../data/mbta_communities/Transit_Station_Areas_Half_Mile_Radius/Transit_Station_Areas_Half_Mile_Radius.shp")
    
print(half_mile_radius_df)

with fs.open("data/gis/WalthamZoning.zip") as f:
    content = BytesIO(f.read())
    zip = ZipFile(content)
    zip.extractall("../../data/gis")
    zoning_df = geo_pd.read_file("../../data/gis/WalthamZoning/WalthamZoning.shp")

print(zoning_df)


                 LOC_ID                                            Address  \
0      F_720576_2954833            726  SOUTH ST, SOUTH ST, WALTHAM, 02453   
1      F_722539_2954565    167  EDGEWATER DR, EDGEWATER DR, WALTHAM, 02453   
2      F_722274_2954887    105  EDGEWATER DR, EDGEWATER DR, WALTHAM, 02453   
3      F_722477_2954895     44  RIVERSIDE DR, RIVERSIDE DR, WALTHAM, 02453   
4      F_722168_2954973     85  KNOLLWOOD DR, KNOLLWOOD DR, WALTHAM, 02453   
...                 ...                                                ...   
13318  F_724424_2970928           17  MILNER ST, MILNER ST, WALTHAM, 02451   
13319  F_723500_2970910           55  CHERYL LN, CHERYL LN, WALTHAM, 02451   
13320  F_731397_2970882       71  RAFFAELE DR, RAFFAELE DR, WALTHAM, 02452   
13321  F_726321_2970932        0  BRAEMORE RD, BRAEMORE RD, WALTHAM, 02451   
13322  F_726015_2970933  295 LOT TRIMOUNT AVE, TRIMOUNT AVE, WALTHAM, 0...   

                                Owner UseCodes  \
0            

In [44]:
# set the waltham zoning crs to match the MassGIS one
zoning_df.to_crs(property_shapefiles_df.crs, inplace=True)

In [45]:
# assign existing parcels to zones
property_shapefiles_df["parcel_geometry"] = property_shapefiles_df["geometry"]
property_shapefiles_df["geometry"] = property_shapefiles_df.centroid
properties_df = property_shapefiles_df.sjoin(zoning_df, how="left", predicate="within")
properties_df.drop(axis="columns", labels=["index_right", "geometry"], inplace=True)

properties_df.rename(columns={"NAME": "ZONE", "parcel_geometry": "geometry"}, inplace=True)

In [47]:
properties_df.at[101, "ZONE"]

'CR'

In [ ]:
# parcels within the 0.5 mi radius of stations
#properties_df["in_range"] = properties_df["geometry"].within(half_mile_radius_df['geometry'])

In [14]:
zoning_df.index

RangeIndex(start=0, stop=170, step=1)

In [12]:
#geo_pd.tools.sjoin(zoning_df, half_mile_radius_df, how="left")

zoning_df.sjoin(half_mile_radius_df, how="left").columns

Index(['NAME', 'CODE', 'SHAPE_LENG', 'ORDINANCE', 'EDITOR', 'CHANGE_DAT',
       'SOURCE', 'PLAN_NAME', 'SHAPE_STAr', 'SHAPE_STLe', 'SHAPE_ST_1',
       'SHAPE_ST_2', 'geometry', 'index_right', 'Shape_Leng', 'Shape_Area'],
      dtype='object')

In [9]:
property_shapefiles_df.columns

Index(['LOC_ID', 'Address', 'Owner', 'UseCodes', 'UseDesc', 'TRANSIT', 'ACRES',
       'SQFT', 'PublicInst', 'NonPubExc', 'Tot_Exclud', 'Tot_Sensit', 'ROW',
       'OpenSpace', 'Hydrology', 'Wetlands', 'TitleV', 'Wellhead1',
       'Flood_SHFA', 'Farmland', 'SurfWatBC', 'Wellhead2', 'IntWellhea',
       'Habitat', 'geometry'],
      dtype='object')